# Using SQLAlchemy to store and retrieve data in a postgres database

In [1]:
from __future__ import division

In [2]:
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# 1 Store housing data from ex1b in a database

Load the data.

In [3]:
data = np.loadtxt('ex1_orig_octave/ex1data2.txt', delimiter=',')

Create a tuple of dictionaries - one dictionary per example in the dataset.

In [4]:
data_as_tuple = tuple([{
    'Id': i, 
    'Size': data[i, 0], 
    'Bedrooms': data[i, 1],
    'Price': data[i, 2]} 
    for i in xrange(len(data))])

Simultaneously create a database called "ng_ml" and an SQLAlchemy engine object.  This is the starting point for SQLAlchemy applications.  
(See http://docs.sqlalchemy.org/en/latest/core/engines.html.)

NOTE:  
A PostgreSQL SERVER MUST BE RUNNING LOCALLY ON YOUR MACHINE, AND THE DATABASE "ng_ml" MUST EXIST.  
TO CREATE THAT DATABASE ON MY MAC OSX LAPTOP FROM THE COMMAND LINE OF A BASH SHELL, I TYPED:

```$ createdb -h localhost -p 5432 -U postgres ng_ml```

In [5]:
eng = create_engine('postgresql:///ng_ml')

Connect to the postgres database and create a table containing the data.

In [6]:
with eng.connect() as con:

    con.execute(text('DROP TABLE IF EXISTS ex1_housing'))
    con.execute(text('''CREATE TABLE ex1_housing(
        Id INTEGER PRIMARY KEY, Size FLOAT, Bedrooms FLOAT, Price FLOAT)'''))
    
    for line in data_as_tuple:
        con.execute(text('''INSERT INTO ex1_housing(Id, Size, Bedrooms, Price) 
            VALUES(:Id, :Size, :Bedrooms, :Price)'''), **line)

# 2 Retrieve data

Query the ex1_housing table in the ng_ml database.

In [14]:
eng = create_engine('postgresql:///ng_ml')
with eng.connect() as con:            
    rs = con.execute(text('SELECT * FROM ex1_housing'))            
    data = np.array(rs.fetchall())[:, 1:] # drop "Id" column from database table.

In [15]:
x0 = data[:, 0] 
x1 = data[:, 1] 
y = data[:, -1]

M = len(y) 

From here, we can continue with the analysis in ex1b_multivariate.ipynb.